In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install torch pandas tqdm scikit-learn
!pip install torch_geometric
#!pip install -e /content/drive/MyDrive/PhD/Study2_review/Published_Codes_Study_2/Production/codes

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.8 MB/s eta 0:00:00


In [ ]:
#need to restart to get this working
#from codes.preprocess import *


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import re

In [ ]:
# prompt: tqdm for pandas

tqdm.pandas()
# Example usage:
# Assuming 'df' is your pandas DataFrame and 'my_function' is a function to apply
#df['new_column'] = df['existing_column'].progress_apply(my_function)
# Replace 'existing_column' and 'my_function' with your actual column and function.
# Example of progress_apply
#df['new_column'] = df['existing_column'].progress_apply(lambda x: x*2)


In [ ]:
'''
The preprocessing pipeline consists of several systematic steps designed to clean, structure, and temporally partition the data for subsequent analysis.
First, only the specified columns are retained, missing values are replaced with empty strings, and duplicate entries (based on a unique tweet identifier) are removed.
Each tweet is then flagged as a “root node” when its tweet ID matches the designated root node value.
To facilitate subsequent graph-based analyses, tweets with a parent ID of zero are re-assigned to reference themselves, ensuring that every tweet has a valid parent relationship.

Next, the news labels are standardized: for the FibVid dataset, “fake” news is recoded as 1 and “real” news as 0 (with the mapping reversed for the TS dataset).
Temporal context is incorporated by mapping each tweet’s root node to its creation date, allowing the calculation of the elapsed time (in minutes) between the creation of each tweet
and that of its root. Tweets with negative elapsed times—i.e., those appearing before their root node—are optionally excluded to preserve temporal consistency.

To capture the progression of information over time, the elapsed time is further binned into several categorical indicators corresponding to
predefined time thresholds (e.g., 0, 1, 15, 20, 60, 90 minutes, and 24 to 48 hours). In particular, the 0-minute category is strictly reserved for the root node,
 while an additional flag is set for tweets beyond 48 hours.

For data partitioning, a time-based, stratified split is performed. Root nodes are grouped by label (fake or real) and sorted chronologically.
Within each group, the oldest 70% of root nodes are allocated to the training set, the subsequent 15% to a validation set, and the most recent 15% to the test set.
This ensures that the training data reflects earlier events, while the validation and test sets capture more recent developments.
To mitigate potential class imbalances, class weights are computed using a balanced weighting strategy based on the training labels.

Finally, the text content is cleaned (for instance, by removing URLs), and a subset of the processed columns is selected.
The resulting dataset is then stored in an SQLite database for further analysis.
'''

'''
The genEgoGraph function constructs an ego graph centered on a specified target node (root node) from a processed tweet dataset.
An ego graph here represents the local network of tweets associated with a given root, including both the tweets themselves and their parent-child relationships.
Below is a detailed explanation of each step within the function:

Filter and Order Nodes:
The function begins by filtering the dataset to include only those tweets that belong to the specified root (i.e., tweets whose root_node field matches the given target).
These nodes are then sorted by the is_root flag in descending order, ensuring that the root node itself appears at the top of the list.

Generate Edge Data:
Next, the function groups the filtered tweets by their tweet_id and parent_id to aggregate the connections between nodes.
The grouping operation counts the frequency of each tweet-to-parent pairing (stored in a column named freq), and then resets the index so that each edge is explicitly represented.
The target root is also inserted as a new column in the edges dataframe, linking each edge to the overall ego graph.

Compile Unique Nodes:
The function creates an initial nodes dataframe containing only the root_node and tweet_id columns. In parallel, it extracts the parent nodes from the edges dataframe
(by taking the parent_id column, renaming it to tweet_id) to ensure that all nodes—whether appearing as a tweet or as a parent—are included.
These two sets of nodes are concatenated, and duplicates (nodes with the same tweet_id) are removed, guaranteeing that each tweet appears only once in the final node list.

Assign Sequential Identifiers:
A sequential identifier (sid) is assigned to each unique node. This numeric index is crucial for later constructing the graph, as it provides a compact representation of the nodes.
The function explicitly checks that the target node (root) is assigned the index 0, ensuring that it occupies a prominent position in the graph.
If the target node is not indexed as 0, an exception is raised.

Map Nodes to Edges:
With the nodes indexed, the function creates a dictionary mapping each tweet ID to its corresponding sid. This mapping is then applied to the edges dataframe:
both the tweet_id and parent_id columns are converted to their numeric indices (named source and target, respectively).
This transformation standardizes the edge data and facilitates downstream graph processing.

Validation and Return:
The function validates that every edge has been successfully mapped to valid source and target indices.
If any mapping fails (resulting in missing indices), an exception is raised. Finally, the function returns two dataframes: one for the nodes (e_nodes) and one for the edges (e_edges),
 both of which are structured for further analysis or visualization of the ego graph.
'''

'\nThe genEgoGraph function constructs an ego graph centered on a specified target node (root node) from a processed tweet dataset.\nAn ego graph here represents the local network of tweets associated with a given root, including both the tweets themselves and their parent-child relationships.\nBelow is a detailed explanation of each step within the function:\n\nFilter and Order Nodes:\nThe function begins by filtering the dataset to include only those tweets that belong to the specified root (i.e., tweets whose root_node field matches the given target).\nThese nodes are then sorted by the is_root flag in descending order, ensuring that the root node itself appears at the top of the list.\n\nGenerate Edge Data:\nNext, the function groups the filtered tweets by their tweet_id and parent_id to aggregate the connections between nodes.\nThe grouping operation counts the frequency of each tweet-to-parent pairing (stored in a column named freq), and then resets the index so that each edge is

In [ ]:
def sortText(x):
    """
    Remove URLs from tweet content
    Args:
      - x (str): input text
    Outputs:
      - x (str): cleaned text
    """
    # remove urls
    x = re.sub('http://\S+|https://\S+', '', x)
    # decode &
    x = x.replace('&amp;', '&')
    x = x.replace("\n", ' ')
    x = ' '.join([y.strip() for y in x.split(' ') if y.strip()])
    return x

def preprocessData(
    df,
    dataset='fibvid',
    columns=[
        'tweet_id', 'root_node', 'parent_id', 'label',
        'tweet_user', 'text', 'target_emotion_stance',
        'create_date'
    ],
    exclude_earlier_tweet=True,
    time_periods=[0, 1, 15, 20, 60, 90, 24 * 60, 48 * 60],
    train_ratio=0.7,
    seed=9898,
    exclude_tweets = True
):
    """
    Args:
      - df (pandas.DataFrame): loaded raw data
      - columns (list): selected columns
      - dataset (str): 'fibvid' or 'ts'
      - exclude_earlier_tweet (bool): whether to exclude tweets created before the root nodes
      - time_periods (list): list of time periods to examine (in mins)
      - train_ratio (int/float): ratio of the train dataset, the rest will be equally split into test and val sets
      - seed (int): random seed
    Ouputs:
      - df (pandas.DataFrame): pre-processed data
    """
    df = df[columns].fillna('').drop_duplicates(subset='tweet_id')
    df['is_root'] = df.apply(lambda x: 1 if x.root_node == x.tweet_id else 0, axis=1)
    # sort parent id -> root nodes -> self-loops when generating ego graphs
    df.parent_id = df.apply(lambda x: x.tweet_id if int(x.parent_id) == 0 else x.parent_id, axis=1)

    # mark fake news as 1
    if dataset == 'fibvid':
        df['label'] = df['label'].map({'fake': 1, 'real': 0})
    elif dataset == 'ts':
        df['label'] = df['label'].map({0: 1, 1: 0})

    # compute time elapsed
    df['root_created'] = df.root_node.map(dict(zip(df.tweet_id, df.create_date)))
    df['time_elapsed'] = (pd.to_datetime(df.create_date) - pd.to_datetime(df.root_created)).dt.total_seconds() / 60

    # exclude tweets created before the root node
    if exclude_tweets is True:
      if len(df[df.time_elapsed < 0]) and exclude_earlier_tweet is True:
          print(f'Excluded {len(df[df.time_elapsed < 0])} tweets created before the root node')
          df = df[df.time_elapsed >= 0]
          print('No. of root nodes', df[df.is_root == 1].tweet_id.nunique())
    else:
      print('Using tweets created before root node: No. of root nodes', df[df.is_root == 1].tweet_id.nunique())

    # convert time_elapsed to categories
    tp_cols = []
    for i in tqdm(time_periods):
        if i <= 90:
            col_name = f'{i}m'
        else:
            col_name = f'{int(i / 60)}h'
        tp_cols.append(col_name)
        df[col_name] = df.time_elapsed.apply(lambda x: 1 if x <= i else 0)

    # make sure 0m contains only the root node
    df['0m'] = df.apply(lambda x: 1 if x.tweet_id == x.root_node else 0, axis=1)
    df['gt_48h'] = 1
    tp_cols.append('gt_48h')


    root_nodes = df[df.is_root == 1].copy()
    root_nodes_sorted = root_nodes.sort_values(by='create_date')  # Sort by creation time (oldest first)
    n = len(root_nodes_sorted)

    # Determine split indices using the provided train_ratio for training,
    # and split the remaining data equally between validation and test.
    # Calculate the number of nodes for each split
    train_count = int(0.7 * n)
    val_count = int(0.15 * n)

    # Create sequential splits
    train_nodes = root_nodes_sorted.iloc[:train_count]
    print(len(train_nodes))
    val_nodes = root_nodes_sorted.iloc[train_count:train_count + val_count]
    test_nodes = root_nodes_sorted.iloc[train_count + val_count:]

    # Build the mapping from each root_node id to its split assignment
    sample_splits = {}
    sample_splits.update(dict(zip(train_nodes.root_node, ['train'] * len(train_nodes))))
    sample_splits.update(dict(zip(val_nodes.root_node, ['val'] * len(val_nodes))))
    sample_splits.update(dict(zip(test_nodes.root_node, ['test'] * len(test_nodes))))


    # Map the splits back to the full dataframe
    df['SPLIT'] = df.root_node.map(sample_splits).fillna('val')

    # Combine training nodes from all groups for computing class weight
    # train_all = pd.concat(train_nodes_list)

    # Compute class weight using the labels from the training set
    w = compute_class_weight(class_weight="balanced", classes=np.array([0, 1]), y=train_nodes.label)
    print('Class weights', w)


    # remove urls form text
    df.text = df.text.progress_apply(sortText)

    cols = [
        'SPLIT', 'tweet_id', 'is_root', 'root_node', 'label',
        'parent_id', 'text', 'target_emotion_stance','create_date','root_created', 'time_elapsed'
    ] + tp_cols
    df = df[cols]

    con = sqlite3.connect(f'{dataset}_data.db')
    df.to_sql('data', con, index=False, if_exists='replace')
    return df

def genEgoGraph(target, data):
    """
    Generate an ego graph centered on a specified target node.

    Args:
      - target (int): ID of the root node (center of the ego graph)
      - data (pandas.DataFrame): Pre-processed tweet data containing at least the
                                 columns 'root_node', 'tweet_id', 'parent_id', and 'is_root'

    Returns:
      - e_nodes (pandas.DataFrame): DataFrame containing unique nodes with assigned sequential IDs (sid)
      - e_edges (pandas.DataFrame): DataFrame containing edges with source and target indices corresponding to e_nodes
    """
    # Filter data to include only tweets that belong to the given root node (target)
    # and sort so that the root node appears at the top (is_root == 1)
    e_nodes = data[data.root_node == target].sort_values(by=['is_root'], ascending=False)

    # Group by tweet and parent IDs to generate edge information,
    # counting the frequency of each tweet-parent pair
    e_edges = e_nodes.groupby(['tweet_id', 'parent_id']).size().to_frame('freq').reset_index()

    # Insert the target root node column into the edges DataFrame for reference
    e_edges.insert(0, 'root_node', target)

    # Retain only the 'root_node' and 'tweet_id' columns for nodes
    e_nodes = e_nodes[['root_node', 'tweet_id']]

    # Extract parent nodes from the edges DataFrame,
    # renaming 'parent_id' to 'tweet_id' so they can be merged with e_nodes
    parent_nodes = e_edges[['root_node', 'parent_id']].rename(columns={'parent_id': 'tweet_id'})

    # Combine the tweet nodes and parent nodes, dropping duplicates to ensure each tweet appears once
    e_nodes = pd.concat([e_nodes, parent_nodes]).drop_duplicates(subset=['tweet_id'], keep='first')

    # Assign a sequential identifier (sid) to each node
    e_nodes.insert(1, 'sid', range(len(e_nodes)))

    # Create a mapping from tweet_id to its assigned sequential ID (sid)
    node_indices = dict(zip(e_nodes.tweet_id, e_nodes.sid))

    # Verify that the target node has been assigned index 0; if not, raise an exception
    if node_indices.get(target) != 0:
        raise Exception(f"{target}: Double check the root node index!!!")

    # Map the tweet_id and parent_id in the edges DataFrame to their corresponding sids
    e_edges['source'] = e_edges.tweet_id.map(node_indices)
    e_edges['target'] = e_edges.parent_id.map(node_indices)

    # Check for any unmapped nodes (resulting in NaN values) in the source or target columns,
    # and raise an exception if mapping failed
    if len(e_edges[(pd.isna(e_edges.source)) | (pd.isna(e_edges.target))]) > 0:
        raise Exception(f"{target}: Mapping failed!!!")

    # Return the nodes and edges DataFrames representing the ego graph
    return e_nodes, e_edges


<>:10: SyntaxWarning: invalid escape sequence '\S'
<>:10: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-2889578126.py:10: SyntaxWarning: invalid escape sequence '\S'
  x = re.sub('http://\S+|https://\S+', '', x)


In [ ]:
# prompt: clear juypter output

from IPython.display import clear_output
clear_output()

In [ ]:
#demo and intuition building

seed = 9898

datasets = {
    'fibvid': [
        '/content/drive/MyDrive/PhD/Study2_review/Copy of claimprop_list_TABSA_finetune_FLAN_pair_train_NER.h5',
        '/content/drive/MyDrive/PhD/Study2_review/Copy of claimprop_list_TABSA_finetune_FLAN_pair_val_NER.h5',
        '/content/drive/MyDrive/PhD/Study2_review/Copy of claimprop_list_TABSA_finetune_FLAN_pair_test_NER.h5'
    ],
    'ts': [
        '/content/drive/MyDrive/PhD/Study2_review/TS/Copy of truth_seeker_NER_clean2.h5'
    ]
}

for dataset, files in datasets.items():
    print(dataset)
    df = pd.concat([
        pd.read_pickle(file, compression='xz').reset_index(drop=True) for file in files
    ]).sort_values(by='create_date').reset_index(drop=True)

    df = preprocessData(
        df,
        dataset=dataset,
        columns=[
            'tweet_id', 'root_node', 'parent_id', 'label',
            'text', 'target_emotion_stance',
            'create_date'
        ],
        exclude_earlier_tweet=True,
        time_periods=[0, 1, 15, 20, 60, 90, 24 * 60, 48 * 60],
        train_ratio=0.7,
        seed=seed
    )

    # Add the `gt_48h` column for time elapsed > 48 hours
    df['gt_48h'] = 1

    display(df)

    #testing
    # Filter to include only root nodes (since the split is done on them)
    root_nodes = df[df.is_root == 1]

    for label, group in root_nodes.groupby('label'):
        train_group = group[group['SPLIT'] == 'train']
        val_group = group[group['SPLIT'] == 'val']
        test_group = group[group['SPLIT'] == 'test']

        # Check temporal ordering within the group
        assert pd.to_datetime(train_group['create_date']).max() <= pd.to_datetime(val_group['create_date']).min(), (
            f"Temporal ordering violated for label {label}: "
            f"latest train date {pd.to_datetime(train_group['create_date']).max()} "
            f"is newer than earliest val date {pd.to_datetime(val_group['create_date']).min()}."
        )
        assert pd.to_datetime(val_group['create_date']).max() <= pd.to_datetime(test_group['create_date']).min(), (
            f"Temporal ordering violated for label {label}: "
            f"latest val date {pd.to_datetime(val_group['create_date']).max()} "
            f"is newer than earliest test date {pd.to_datetime(test_group['create_date']).min()}."
        )


    print("Time-based splitting is correctly maintained.")

    # Include `gt_48h` in the targets and print value counts
    targets = df[df.is_root == 1]
    print(targets.groupby('SPLIT').label.value_counts())

    # Generate ego graphs including the new time period
    print('Start to generate ego graphs...')
    time_periods = ['0m', '1m', '15m', '20m', '60m', '90m', '24h', '48h', 'gt_48h']
    try:
      for te_idx, te in enumerate(time_periods):
            # Select data for the current and all earlier time periods
            cumulative_periods = time_periods[:te_idx + 1]
            te_data = df[df[cumulative_periods].any(axis=1)]
            # Ensure the text of the parent ID is captured
            te_data = te_data[(te_data.root_node.isin(targets.tweet_id)) & (te_data.parent_id.isin(df.tweet_id))]
            # Generate ego graphs
            con = sqlite3.connect(f'{dataset}_ego_graphs.db') #this will be saved in the content directory, hence need to shift it
            for i in tqdm(range(len(targets))):
                target = targets.tweet_id.iloc[i]
                e_nodes, e_edges = genEgoGraph(target, te_data)
                if_exists = 'append' if i > 0 else 'replace'
                e_nodes.to_sql(f'nodes_{te}', con, index=False, if_exists=if_exists)
                e_edges.to_sql(f'edges_{te}', con, index=False, if_exists=if_exists)
    except Exception as e:
      print('\n')
      print(e)
      print('te:',te)
      print('te_data:',te_data)
      efg

    print('-------------------------')







fibvid
Excluded 3262 tweets created before the root node
No. of root nodes 1425


100%|██████████| 8/8 [00:00<00:00, 24.97it/s]


997
Class weights [2.13948498 0.65248691]


100%|██████████| 83910/83910 [00:00<00:00, 127222.70it/s]


,SPLIT,tweet_id,is_root,root_node,label,parent_id,text,target_emotion_stance,create_date,root_created,time_elapsed,0m,1m,15m,20m,60m,90m,24h,48h,gt_48h
0,train,1223618900044152834,1,1223618900044152834,1,1223618900044152834,"First Sen. Warren, & now Speaker Pelosi questi...",neutral emotion & AGAINST stance towards Warre...,2020-02-01T14:47:34.000Z,2020-02-01T14:47:34.000Z,0.000000,1,1,1,1,1,1,1,1,1
1,train,1223789646989144065,1,1223789646989144065,1,1223789646989144065,Statement on the final installment of the CNN/...,neutral emotion & NEUTRAL stance towards CNN;n...,2020-02-02T02:06:03.000Z,2020-02-02T02:06:03.000Z,0.000000,1,1,1,1,1,1,1,1,1
2,train,1223789928192057346,0,1223789646989144065,1,1223789646989144065,@NateSilver538,neutral emotion & FAVOR stance towards @NateSi...,2020-02-02T02:07:10.000Z,2020-02-02T02:06:03.000Z,1.116667,0,0,1,1,1,1,1,1,1
3,train,1223789991798636544,0,1223789646989144065,1,1223789646989144065,that respondents name was Shmoe Shmiden,neutral emotion & NEUTRAL stance towards Shmoe...,2020-02-02T02:07:25.000Z,2020-02-02T02:06:03.000Z,1.366667,0,0,1,1,1,1,1,1,1
4,train,1223790057133346816,0,1223789646989144065,1,1223789646989144065,Read: Bernie polled too highly and they looked...,neutral emotion & AGAINST stance towards Bernie,2020-02-02T02:07:41.000Z,2020-02-02T02:06:03.000Z,1.633333,0,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87167,train,1344341225508401152,0,1257264160213217285,1,1341394949657518080,@RepMoBrooks @Robert_Aderholt @RepMikeRogersAL...,anger emotion & NEUTRAL stance towards @TTuber...,2020-12-30T17:54:59.000Z,2020-05-04T11:01:49.000Z,346013.166667,0,0,0,0,0,0,0,0,1
87168,train,1344397515383320581,0,1257264160213217285,1,1341118726901542917,INSANITY! PAY CLOSE ATTENTION PEOPLE THAT ARE ...,anger emotion & NEUTRAL stance towards AWAKE,2020-12-30T21:38:40.000Z,2020-05-04T11:01:49.000Z,346236.850000,0,0,0,0,0,0,0,0,1
87169,train,1344416282314203141,0,1264558926021959680,1,1340208320951562240,@GeraldoRivera @realDonaldTrump,neutral emotion & NEUTRAL stance towards @real...,2020-12-30T22:53:14.000Z,2020-05-24T14:08:36.000Z,317324.633333,0,0,0,0,0,0,0,0,1
87170,train,1344434240432336896,0,1264558926021959680,1,1340208320951562240,"Fight for the greatest leader of our lifetime,...",neutral emotion & FAVOR stance towards Donald ...,2020-12-31T00:04:35.000Z,2020-05-24T14:08:36.000Z,317395.983333,0,0,0,0,0,0,0,0,1


...

100%|██████████| 103672/103672 [00:01<00:00, 66850.99it/s]


,SPLIT,tweet_id,is_root,root_node,label,parent_id,text,target_emotion_stance,create_date,root_created,time_elapsed,0m,1m,15m,20m,60m,90m,24h,48h,gt_48h
0,train,46516,0,378,0,378,I might get an interview with Mike Huckabee ab...,neutral emotion & FAVOR stance towards Mike Hu...,2008-05-05 15:20:48,2008-05-05 15:20:48,0.000000e+00,0,1,1,1,1,1,1,1,1
1,train,378,1,378,0,378,Says Mike Huckabee appeared in diabetes infome...,neutral emotion & AGAINST stance towards Mike ...,2008-05-05 15:20:48,2008-05-05 15:20:48,0.000000e+00,1,1,1,1,1,1,1,1,1
2,train,46541,0,378,0,378,@dporter THANKS! I may get to interview Mike H...,neutral emotion & FAVOR stance towards @dporte...,2008-05-06 12:27:06,2008-05-05 15:20:48,1.266300e+03,0,0,0,0,0,0,1,1,1
3,train,88,1,88,0,88,"""The law says that mental health must be treat...",neutral emotion & NEUTRAL stance towards,2008-10-01 17:11:31,2008-10-01 17:11:31,0.000000e+00,1,1,1,1,1,1,1,1,1
4,train,13397,0,88,0,88,pages upon pages of provision to require insur...,neutral emotion & NEUTRAL stance towards,2008-10-01 17:11:31,2008-10-01 17:11:31,0.000000e+00,0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103667,test,133014,0,1033,1,1033,Countries should be wanting to leave the Unite...,neutral emotion & AGAINST stance towards the U...,2022-02-28 22:17:26,2021-09-18 16:14:20,2.350831e+05,0,0,0,0,0,0,0,0,1
103668,test,127838,0,1005,1,1005,@ragedcan @LBJ20Trey @duncTTV @KDTreeey5 @TheN...,anger emotion & NEUTRAL stance towards @LBJ20T...,2022-02-28 22:22:27,2022-02-07 21:39:10,3.028328e+04,0,0,0,0,0,0,0,0,1
103669,test,132719,0,1033,1,1033,"... avoided, and prevented, and you spare the ...",neutral emotion & AGAINST stance towards the u...,2022-02-28 22:32:47,2021-09-18 16:14:20,2.350985e+05,0,0,0,0,0,0,0,0,1
103670,test,133156,0,1033,1,1033,"... control. At that point, KNOW AND UNDERSTAN...",neutral emotion & AGAINST stance towards the u...,2022-02-28 22:34:36,2021-09-18 16:14:20,2.351003e+05,0,0,0,0,0,0,0,0,1


Time-based splitting is correctly maintained.
SPLIT  label
test   1        114
       0         19
train  0        412
       1        201
val    1        116
       0         15
Name: count, dtype: int64
Start to generate ego graphs...


100%|██████████| 877/877 [00:28<00:00, 30.76it/s]

-------------------------


In [ ]:
df.SPLIT.value_counts()

,count
SPLIT,
train,59099
test,27132
val,17441


In [ ]:
#checking SQL
con = sqlite3.connect(f'{dataset}_ego_graphs.db')
cursor = con.cursor()
cursor.execute(f"SELECT * FROM nodes_{te} LIMIT 5")
print(cursor.fetchall())  # Displays a few rows of the stored nodes
cursor.execute(f"SELECT * FROM edges_{te} LIMIT 5")
print(cursor.fetchall())  # Displays a few rows of the stored edges
con.close()

[(378, 0, 378), (378, 1, 46509), (378, 2, 46436), (378, 3, 46504), (378, 4, 46440)]
[(378, 378, 378, 1, 0, 0), (378, 46277, 378, 1, 140, 0), (378, 46278, 378, 1, 46, 0), (378, 46279, 378, 1, 208, 0), (378, 46280, 378, 1, 134, 0)]


In [ ]:
import sqlite3
import pandas as pd

def load_ego_graph_from_sql(root_id, time_slice='gt_48h', dataset='fibvid'):
    con = sqlite3.connect(f'{dataset}_ego_graphs.db')

    nodes_df = pd.read_sql_query(f"""
        SELECT * FROM nodes_{time_slice}
        WHERE root_node = ?
    """, con, params=(root_id,))

    edges_df = pd.read_sql_query(f"""
        SELECT * FROM edges_{time_slice}
        WHERE root_node = ?
    """, con, params=(root_id,))

    con.close()
    return nodes_df, edges_df


root_id = '572'  # replace with any root_node of interest
nodes, edges = load_ego_graph_from_sql(root_id, dataset='ts')

print(f"📌 Ego graph for root_node: {root_id}")
print(f"Number of nodes: {len(nodes)}")
print(f"Number of edges: {len(edges)}")

print("\n🔹 Sample nodes:")
display(nodes.head())

print("\n🔹 Sample edges:")
display(edges.head())


📌 Ego graph for root_node: 572
Number of nodes: 5
Number of edges: 5

🔹 Sample nodes:


,root_node,sid,tweet_id
0,572,0,572
1,572,1,72685
2,572,2,72682
3,572,3,72683
4,572,4,72684



🔹 Sample edges:


,root_node,tweet_id,parent_id,freq,source,target
0,572,572,572,1,0,0
1,572,72682,572,1,2,0
2,572,72683,572,1,3,0
3,572,72684,572,1,4,0
4,572,72685,572,1,1,0


In [ ]:
file = '/content/drive/MyDrive/PhD/Study2_review/TS/Copy of truth_seeker_NER_clean2.h5'
df = pd.concat([
    pd.read_pickle(file, compression='xz').reset_index(drop=True) for file in files
])
df[df.root_node==572]

,author,statement,BinaryNumTarget,tweet,timestamp,PARENT_ID,TWEET_ID,root_node,text,dict_target_emotion_stance,target_emotion_stance,parent_id,tweet_id,create_date,label,time_elapsed
52390,Madison Czopek,Cuomo opens New York 72 hours before his daugh...,0.0,Cuomo basically opened up New York 72 hours be...,2021-06-16 17:26:19,572,72685,572,Cuomo basically opened up New York 72 hours be...,"[{'target': 'Cuomo', 'emotion': 'neutral', 'st...",neutral emotion & FAVOR stance towards Cuomo;n...,572,72685,2021-06-16 17:26:19,0,0.000000
52391,Madison Czopek,Cuomo opens New York 72 hours before his daugh...,0.0,Cuomo opens New York 72 hours before his daugh...,2021-06-16 17:26:19,0,572,572,Cuomo opens New York 72 hours before his daugh...,"[{'target': 'Cuomo', 'emotion': 'neutral', 'st...",neutral emotion & NEUTRAL stance towards Cuomo...,0,572,2021-06-16 17:26:19,0,0.000000
52392,Madison Czopek,Cuomo opens New York 72 hours before his daugh...,0.0,Andrew Cuomo opened New York 72 hours before h...,2021-06-18 13:41:41,572,72682,572,Andrew Cuomo opened New York 72 hours before h...,"[{'target': 'Andrew Cuomo', 'emotion': 'anger'...",anger emotion & AGAINST stance towards Andrew ...,572,72682,2021-06-18 13:41:41,0,2655.366667
52393,Madison Czopek,Cuomo opens New York 72 hours before his daugh...,0.0,@KHerriage @RyanAFournier Cuomo says NY is at ...,2021-06-18 14:49:38,572,72683,572,@KHerriage @RyanAFournier Cuomo says NY is at ...,"[{'target': 'Cuomo', 'emotion': 'neutral', 'st...",neutral emotion & AGAINST stance towards Cuomo...,572,72683,2021-06-18 14:49:38,0,2723.316667
52394,Madison Czopek,Cuomo opens New York 72 hours before his daugh...,0.0,@jobpings Cuomo opens New York 72 hours before...,2021-12-29 21:35:27,572,72684,572,@jobpings Cuomo opens New York 72 hours before...,"[{'target': '@jobpings Cuomo', 'emotion': 'neu...",neutral emotion & NEUTRAL stance towards @jobp...,572,72684,2021-12-29 21:35:27,0,282489.133333


In [ ]:
con = sqlite3.connect(f'{dataset}_data.db')

# List all tables and preview their contents
tables = con.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
for table in tables:
    table_name = table[0]
    print(f"Contents of Table: {table_name}")
    df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5;", con)
    display(df)
    print("\n")

con.close()

Contents of Table: data


,SPLIT,tweet_id,is_root,root_node,label,parent_id,text,target_emotion_stance,create_date,root_created,time_elapsed,0m,1m,15m,20m,60m,90m,24h,48h,gt_48h
0,train,46516,0,378,0,378,I might get an interview with Mike Huckabee ab...,neutral emotion & FAVOR stance towards Mike Hu...,2008-05-05 15:20:48,2008-05-05 15:20:48,0.0,0,1,1,1,1,1,1,1,1
1,train,378,1,378,0,378,Says Mike Huckabee appeared in diabetes infome...,neutral emotion & AGAINST stance towards Mike ...,2008-05-05 15:20:48,2008-05-05 15:20:48,0.0,1,1,1,1,1,1,1,1,1
2,train,46541,0,378,0,378,@dporter THANKS! I may get to interview Mike H...,neutral emotion & FAVOR stance towards @dporte...,2008-05-06 12:27:06,2008-05-05 15:20:48,1266.3,0,0,0,0,0,0,1,1,1
3,train,88,1,88,0,88,"""The law says that mental health must be treat...",neutral emotion & NEUTRAL stance towards,2008-10-01 17:11:31,2008-10-01 17:11:31,0.0,1,1,1,1,1,1,1,1,1
4,train,13397,0,88,0,88,pages upon pages of provision to require insur...,neutral emotion & NEUTRAL stance towards,2008-10-01 17:11:31,2008-10-01 17:11:31,0.0,0,1,1,1,1,1,1,1,1


In [ ]:
df[df['tweet_id']==1223618900044152834]

,SPLIT,tweet_id,is_root,root_node,label,parent_id,text,target_emotion_stance,create_date,root_created,time_elapsed,0m,1m,15m,20m,60m,90m,24h,48h,gt_48h


The numbers in your SQLite query result are identifiers for nodes and their relationships in the ego graph. Let’s break it down more clearly:

SQL Result:
plaintext
Copy code
[(378, 0, 378), (378, 1, 46516), (378, 2, 46541), (88, 0, 88), (88, 1, 13397)]
These numbers correspond to the structure of the ego graph generated by the genEgoGraph function.

Breaking It Down:
Each tuple represents a single edge or relationship in the ego graph and contains the following:

root_node (First Number):

The ID of the central or "ego" node for which the graph is being built.
Example:
In (378, 0, 378), 378 is the root node.
In (88, 1, 13397), 88 is the root node.
source (Second Number):

The sid (sequential ID) of the source node in the ego graph.
This is generated by the line in the code:
python
Copy code
e_nodes.insert(1, 'sid', range(len(e_nodes)))
The sid ensures each node in the ego graph has a unique and sequential index.
target (Third Number):

The sid of the target node (neighbor) in the ego graph.
This is mapped from the tweet_id and parent_id relationships in e_edges:
python
Copy code
e_edges['source'] = e_edges.tweet_id.map(node_indices)
e_edges['target'] = e_edges.parent_id.map(node_indices)
Example Interpretations:
(378, 0, 378):

Root Node: 378
Source Node (sid): 0 (the root node itself)
Target Node (sid): 378 (self-loop on the root node)
(378, 1, 46516):

Root Node: 378
Source Node (sid): 1 (a neighbor of the root node)
Target Node (sid): 46516 (another node connected to the source node)

In [ ]:
#data validation

In [ ]:
# Moving the dbs to a folder

import shutil

# Define the source and destination paths
source_path = "/content/fibvid_ego_graphs.db"
destination_folder = "/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data" # Replace with your desired folder

# Create the destination folder if it doesn't exist
import os
os.makedirs(destination_folder, exist_ok=True)

destination_path = os.path.join(destination_folder, "fibvid_ego_graphs.db")

# Move the file
shutil.move(source_path, destination_path)


# Define the source and destination paths
source_path = "/content/fibvid_data.db"
destination_folder = "/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data" # Replace with your desired folder

# Create the destination folder if it doesn't exist
import os
os.makedirs(destination_folder, exist_ok=True)

destination_path = os.path.join(destination_folder, "fibvid_data.db")

# Move the file
shutil.move(source_path, destination_path)


# Define the source and destination paths
source_path = "/content/ts_data.db"
destination_folder = "/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data" # Replace with your desired folder

# Create the destination folder if it doesn't exist
import os
os.makedirs(destination_folder, exist_ok=True)

destination_path = os.path.join(destination_folder, "ts_data.db")

# Move the file
shutil.move(source_path, destination_path)

# Define the source and destination paths
source_path = "/content/ts_ego_graphs.db"
destination_folder = "/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data" # Replace with your desired folder

# Create the destination folder if it doesn't exist
import os
os.makedirs(destination_folder, exist_ok=True)

destination_path = os.path.join(destination_folder, "ts_ego_graphs.db")

# Move the file
shutil.move(source_path, destination_path)


'/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/ts_ego_graphs.db'